In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

# Pandas 2
## 집계와 분류
하나의 값으로 대용량 데이터세트의 기본 특성에 대한 통찰력을 제공

### Pandas의 간단한 집계 연산

> Series의 경우

In [2]:
# 1. 랜덤 샘플을 생성하자.
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
display(ser)

# 2. 내부적으로 선언된 집계 함수를 사용하자.
display(ser.sum())
display(ser.mean())

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

2.8119254917081569

0.5623850983416314

> Pandas의 경우

In [3]:
# 1. 랜덤 샘플을 생성하자.
df = pd.DataFrame({'A': rng.rand(5),'B': rng.rand(5)})
display(df)

# 2. 내부적으로 선언된 집계 함수를 사용하자.
display(df.mean())

# 3. 각 row에 대한 평균을 내보자.
display(df.mean(axis='columns'))

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


A    0.477888
B    0.443420
dtype: float64

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

> 파이썬에서 제공하는 data를 사용해보자.

In [4]:
import seaborn as sns
planets = sns.load_dataset('planets')

# 데이터의 형상을 보자
display(planets.shape)

# 데이터의 샘플을 보자
display(planets.head())

# 데이터의 집계값을 간단하게 살펴보자.
display(planets.dropna().describe())

(1035, 6)

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


### GroupBy : 분할, 적용, 결합
- 분할 : 지정된 키 값을 기준으로 DataFrame을 나누고 분류하는 단계다.
- 적용 : 개별 그룹 내에서 일반적으로 집계, 변환, 필터링 같은 함수를 계산한다.
- 결합 : 이 연산의 결과를 결과 배열에 병합한다.

> **분할, 적용, 결합**

In [5]:
# 1. DataFrame을 생성한다.
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'], 
                   'data': range(6)},
                 columns = ['key', 'data'])

display(df)

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [6]:
# 2. Key를 기준으로 groupby를 하자.
df.groupby('key')

In [7]:
# 3. key를 기준으로 sum을 하자.
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


> **GourpBy 객체**

In [8]:
# 1. 파이썬에서 제공하는 데이터를 다시 한 번 보자. method를 key로 사용한다.
planets.groupby('method').median()

,number,orbital_period,mass,distance,year
method,,,,,
Astrometry,1.0,631.180000,NaN,17.875,2011.5
Eclipse Timing Variations,2.0,4343.500000,5.125,315.360,2010.0
Imaging,1.0,27500.000000,NaN,40.395,2009.0
Microlensing,1.0,3300.000000,NaN,3840.000,2010.0
Orbital Brightness Modulation,2.0,0.342887,NaN,1180.000,2011.0
Pulsar Timing,3.0,66.541900,NaN,1200.000,1994.0
Pulsation Timing Variations,1.0,1170.000000,NaN,NaN,2007.0
Radial Velocity,1.0,360.200000,1.260,40.445,2009.0
Transit,1.0,5.714932,1.470,341.000,2012.0


In [9]:
# 2. 'orbital_period' column을 선택
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [10]:
# 3. GroupBy 객체는 그룹을 직접 순회할 수 있다.
# 각 메서드에 대하여 속하는 그룹을 리턴한다.
for (method, group) in planets.groupby('method'):
    print "{0:30s} shape={1}".format(method, group.shape)

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


In [11]:
# 4. 각 group을 설명하는 일련의 집계 연산을 수행할 수 있다.
# groupby 후, year 컬럼에 대해서 일련의 집계 연산을 수행하는 메소드.
# 멀티인덱스를 row, col으로 나누어주거나, row, col을 합쳐서 멀티 인덱스를 만들어준다.
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.000000
       Eclipse Timing Variations           9.000000
       Imaging                            38.000000
       Microlensing                       23.000000
       Orbital Brightness Modulation       3.000000
       Pulsar Timing                       5.000000
       Pulsation Timing Variations         1.000000
       Radial Velocity                   553.000000
       Transit                           397.000000
       Transit Timing Variations           4.000000
mean   Astrometry                       2011.500000
       Eclipse Timing Variations        2010.000000
       Imaging                          2009.131579
       Microlensing                     2009.782609
       Orbital Brightness Modulation    2011.666667
       Pulsar Timing                    1998.400000
       Pulsation Timing Variations      2007.000000
       Radial Velocity                  2007.518987
       Transit             

> **집계, 필터, 변환, 적용**

In [12]:
# 랜덤으로 DataFrame을 생성한다.
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                  'data1': range(6),
                  'data2': rng.randint(0, 10, 6),},
                 columns = ['key', 'data1', 'data2'])

df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [13]:
"""집계 : aggregate()""" 

# 1) 직접 함수를 리스트의 원소로 넣어도 되고, 문자열로 넣어도 된다.
# data1/data2 에 동시에 적용
display(df.groupby('key').aggregate(['min', np.median, max]))

# 2) 딕셔너리를 전달하는 경우. data에 대해서 각각 지정이 가능
display(df.groupby('key').aggregate(
        {'data1': [min, np.mean], 'data2': 'max'}))

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

data1      data2
      min mean   max
key                 
A       0  1.5     5
B       1  2.5     7
C       2  3.5     9

In [14]:
"""필터링 : filter(), 그룹 속성을 기준으로 데이터를 걸러낼 수 있다."""

# std가 4보다 큰 그룹을 filter 한다.
def filter_func(x):
    return x['data2'].std() > 4

display(df)
display(df.groupby('key').std())

# 그룹 B, C에 속한 데이터만 return 할 것이다. filter 함수는 부울 값을 반환한다.
display(df.groupby('key').filter(filter_func))
display(df.groupby('key').filter(lambda x: x['data2'].std() > 4))

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


,data1,data2
key,,
A,2.12132,1.414214
B,2.12132,4.949747
C,2.12132,4.242641


,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


In [15]:
"""변환 : trnasformation"""

display(df)

# 그룹에 따른 평균을 구한다. 
# 즉 그룹 A의 data1, data2의 평균을 각각 구하고 데이터에서 평균을 뺀다.
# 정규화에 사용되면 좋을 것이다.
display(df.groupby('key').transform(lambda x: x - x.mean()))

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [16]:
"""apply - 임의의 함수를 그룹 결과에 적용할 때 사용"""

# 같은 그룹에 속하는 data2의 합으로 data1의 값을 각각 나눈다.
def norm_by_data2(x):
    # x는 그룹 값을 가지는 DataFrame
    x['data1'] /= x['data2'].sum()
    return x

display(df)
display(df.groupby('key').apply(norm_by_data2))

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3
3,A,0.375000,3
4,B,0.571429,7
5,C,0.416667,9


> **분할 Key를 따로 지정**
- 하나의 열 이름을 기준으로 DataFrame을 분할하였는데, 분할키를 지정하는 방식을 살펴보자

In [17]:
# 1. 분할키를 리스트로 지정한다.
L = [0, 1, 0, 1, 2, 0]
display(df)

# 0 - index 0, 2, 5 원소의 sum
# 1 - index 1, 3 원소의 sum
# 2 - index 4 원소의 sum
display(df.groupby(L).sum())

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


,data1,data2
0,7,17
1,4,3
2,4,7


In [18]:
# 2. 분할키를 컬럼이름이 아닌 DataFrame을 직접 줄 수도 있다.
df.groupby(df['key']).sum()

,data1,data2
key,,
A,3,8
B,5,7
C,7,12


In [19]:
# 3. 인덱스 값을 그룹 키에 매핑하는 딕셔너리를 제공
# 키를 지정한다.
df2 = df.set_index('key')
display(df2)

# A는 vowel로 매핑되고, B, C는 consonant로 묶여서 매핑된다.
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}

display(df2.groupby(mapping).sum())

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


,data1,data2
consonant,12,19
vowel,3,8


In [20]:
# 4. 파이썬 함수를 전달한다.
df2.groupby(str.lower).mean()

,data1,data2
a,1.5,4.0
b,2.5,3.5
c,3.5,6.0


> **분류(Grouping) 문제**

In [21]:
# year를 10으로 나눈다 : 세기를 정하려는 것.
decade = 10 * (planets['year'] // 10)
# decade의 이름
decade = decade.astype(str) + 's'

# decade의 name key에 접근
decade.name = 'decade'

# 각 연도구간별로 행성이 발견된 횟수를 구할 수 있다.
display(planets.groupby(['method', decade])['number'].sum())
display(planets.groupby(['method', decade])['number'].sum().unstack().fillna(0))

method                         decade
Astrometry                     2010s       2
Eclipse Timing Variations      2000s       5
                               2010s      10
Imaging                        2000s      29
                               2010s      21
Microlensing                   2000s      12
                               2010s      15
Orbital Brightness Modulation  2010s       5
Pulsar Timing                  1990s       9
                               2000s       1
                               2010s       1
Pulsation Timing Variations    2000s       1
Radial Velocity                1980s       1
                               1990s      52
                               2000s     475
                               2010s     424
Transit                        2000s      64
                               2010s     712
Transit Timing Variations      2010s       9
Name: number, dtype: int64

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


## 피벗 테이블
- 피벗 테이블은 입력값으로 간단한 열 단위의 데이터를 취하고, 그 데이터에 대한 다차원 요약을 제공하는 2차원 테이블로 항목을 그룹핑한다.
- 피벗 테이블은 근본적으로 GroupBy 집계의 다차원 버전이다.

In [22]:
# 1. 타이타닉의 승객 데이터베이스를 사용할 것이다.
import seaborn as sns
titanic = sns.load_dataset('titanic')

# 2. 타이타닉 데이터의 샘플을 보자.
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [23]:
# 3. 성별에 따른 생존율을 GroupBy 연산을 사용하여 구해보자.
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


In [24]:
# 4. 성별과 좌석 등급별 생존율을 GroupBy 연산을 사용하여 구해보자.
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [25]:
# 5. 피벗 테이블을 사용하여 4번에서 구했던 내용을 간단하게 구해보자.
titanic.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


> **다단계 피벗 테이블**

In [26]:
# 1. 연령을 추가하여 성별, 좌석 등급, 연령의 3차원으로 보자. 연령은 2개의 구간으로 나누었다.
age = pd.cut(titanic['age'], [0, 18, 80])

# 3차원 테이블이다.
display(titanic.pivot_table('survived', ['sex', age], 'class'))

# sum
display(titanic.pivot_table('survived', 
                            ['sex', age], # index
                            'class', # column
                            aggfunc=np.sum))

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

class            First  Second  Third
sex    age                           
female (0, 18]      10      14     22
       (18, 80]     72      54     25
male   (0, 18]       4       9     11
       (18, 80]     36       6     27

In [27]:
# 2. 열 기준으로 동작할 때는 pd.qcut 함수를 사용한다. 임으로 2개로 나누었다.
fare = pd.qcut(titanic['fare'], 2)

# 4차원 테이블이다.
titanic.pivot_table('survived', 
                    ['sex', age], # index
                    [fare, 'class']) # column

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

> **기타 피벗 테이블 옵션**

In [28]:
# 1. aggfunc 옵션
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived': sum, 'fare': 'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

In [29]:
# 2. 그룹별 총합을 계산하는 옵션
titanic.pivot_table('survived', 
                    index='sex', 
                    columns='class', 
                    margins=True) # default : mean

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


## 벡터화된 문자열 연산

### Pandas 문자열 연산
- Pandas는 문자열을 담고 있는 Pandas Series와 Index 객체의 `str` 속성을 통해 벡터화된 문자열 연산을 수행하고 누락된 데이터를 올바르게 처리하기 위한 기능을 제공

In [30]:
# 1. 테스트를 위한 데이터 세트 생성
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
names = pd.Series(data)

names

0    peter
1     Paul
2     None
3     MARY
4    gUIDO
dtype: object

In [31]:
# 2. 누락된 값은 건너뛰면서 모든 항목의 첫 글자를 대문자로 변경하는 메소드를 호출
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

> **문자열 메서드들**

In [32]:
# 테스트를 위한 데이터 세트 생성
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                  'Eric Idle', 'Terry Jones', 'Michael Palin'])

In [33]:
"""lower()"""

monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

In [34]:
"""len()"""

monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

In [35]:
"""startswith()"""

monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

In [36]:
"""split()"""

display(monte.str.split())
display(monte.str.split().str.get(-1))

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

In [37]:
"""슬라이싱"""

monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

> **정규 표현식을 활용하는 메서드**

In [38]:
# 각 요소의 시작 문자와 붙어 있는 그룹을 요청해 각 요소로부터 이름 부분을 추출
monte.str.extract('([A-Za-z]+)')

/Users/yoonjeonghun/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

In [39]:
# 문자열 시작(^)과 문자열 끝($)을 나타내는 정규 표현식을 사용해 자음으로 시작하고 끝나는 모든 이름을 찾아보자.
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

## 시계열 다루기
- 타임스탬프 : 특정 시점을 말한다.
- 시간 간격, 기간 : 특정 시작점과 종료점 사이의 시간의 길이를 말한다.
- 시간 델타(time delta)나 지속 기간(duration) : 정확한 시간 길이를 말한다.

### 파이썬에서의 날짜와 시간

> **기본 파이썬 날짜와 시간: datetime, dateutil**

In [40]:
from datetime import datetime
datetime(year=2018, month=2, day=4)

datetime.datetime(2018, 2, 4, 0, 0)

In [41]:
# 날짜를 해석할 수 있다.
from dateutil import parser
date = parser.parse("4th of July, 2015")

date

datetime.datetime(2015, 7, 4, 0, 0)

In [42]:
# 요일을 출력하는 작업
date.strftime('%A')

'Saturday'

### 타입이 지정된 시간 배열: NuPy의 datetime64
- 64비트 정밀도에 제한되기 때문에 이 유형의 연산이 파이썬의 datetime 객체로 직접 작업하는 것보다 훨씬 더 빨리 수행될 수 있다.
- 시간 부해능과 최대 시간 사이의 절충점을 도입한다.

In [43]:
import numpy as np
date = np.array('2018-02-04', dtype=np.datetime64)

date

array(datetime.date(2018, 2, 4), dtype='datetime64[D]')

In [44]:
# 주어진 날짜로부터 12일 후까지의 날짜를 가지고 온다.
date + np.arange(12)

array(['2018-02-04', '2018-02-05', '2018-02-06', '2018-02-07',
       '2018-02-08', '2018-02-09', '2018-02-10', '2018-02-11',
       '2018-02-12', '2018-02-13', '2018-02-14', '2018-02-15'], dtype='datetime64[D]')

### Pandas에서의 날짜와 시: 두 세계의 최선
- datetime과 dateutil의 사용 편의성과 numpy.datetime64의 효율적인 저장소와 벡터화된 인터페이스를 결합

In [45]:
date = pd.to_datetime("4th of July, 2015")
date

Timestamp('2015-07-04 00:00:00')

In [46]:
date.strftime('%A')

'Saturday'

In [47]:
date + pd.to_timedelta(np.arange(12), 'D')

DatetimeIndex(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
               '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
               '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
              dtype='datetime64[ns]', freq=None)

## Pandas 시계열: 시간으로 인덱싱하기

In [48]:
# 타임스탬프로 데이터를 인덱싱하기
index = pd.DatetimeIndex(['2014-07-04', 
                          '2014-08-04', 
                          '2015-07-04', 
                          '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)

display(data)

# 타임스탬프를 이용하여 슬라이싱 하기
display(data['2014-07-04':'2015-07-04'])

# 연도수를 이용하여 데이터를 불러오기
display(data['2015'])

2014-07-04    0
2014-08-04    1
2015-07-04    2
2015-08-04    3
dtype: int64

2014-07-04    0
2014-08-04    1
2015-07-04    2
dtype: int64

2015-07-04    2
2015-08-04    3
dtype: int64

### DatetimeIndex를 생성하기

In [49]:
# 1. 다양한 형태로 datetime index를 생성해보자.
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])

dates

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
               '2015-07-08'],
              dtype='datetime64[ns]', freq=None)

In [50]:
# 2. 함수에 주기 코드를 추가해 PeriodIndex로 전환
dates.to_period('D')

PeriodIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
             '2015-07-08'],
            dtype='period[D]', freq='D')

In [51]:
# 3. 어떤 날자에서 다른 날짜를 빼보자.
dates - dates[0]

TimedeltaIndex(['0 days', '1 days', '3 days', '4 days', '5 days'], dtype='timedelta64[ns]', freq=None)

### 정규 시퀀스 : pd.date_range()

In [52]:
# 시작점과 종료점을 준다.
display(pd.date_range('2015-07-03', '2015-07-10'))

# 시작점과 기간을 준다.
display(pd.date_range('2015-07-03', periods=8))

# 시작점, 기간 및 단위를 준다.
display(pd.date_range('2015-07-03', periods=8, freq='H'))

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06',
               '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06',
               '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2015-07-03 00:00:00', '2015-07-03 01:00:00',
               '2015-07-03 02:00:00', '2015-07-03 03:00:00',
               '2015-07-03 04:00:00', '2015-07-03 05:00:00',
               '2015-07-03 06:00:00', '2015-07-03 07:00:00'],
              dtype='datetime64[ns]', freq='H')

## 고성능 Pandas: eval()과 query()
- NumPy의 벡터화/브로드캐스팅된 연산과 Pandas의 그룹화 유형의 연산은 일반적인 경우에 대체로 효율적이고 효과적이기는 하지만 임시 중간 객체 생성에 의존하는 경우가 종종 있어 계산 시관과 메모리 사용에 과도한 오버헤드를 일으킬 수 있다.
- 2014년 1월에 출시된 0.13 버전을 기준으로 Pandas는 비용이 많이 드는 중간 배열의 할당 없이 속도가 빠른 C연산에 직접 접근할 수 있는 실험적인 도구

### eval()

In [53]:
# 1. 테스트 셋을 만들어보자.
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) 
                      for i in range(4))

display(df1)
display(df2)
display(df3)
display(df4)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.374540,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,...,0.119594,0.713245,0.760785,0.561277,0.770967,0.493796,0.522733,0.427541,0.025419,0.107891
1,0.031429,0.636410,0.314356,0.508571,0.907566,0.249292,0.410383,0.755551,0.228798,0.076980,...,0.093103,0.897216,0.900418,0.633101,0.339030,0.349210,0.725956,0.897110,0.887086,0.779876
2,0.642032,0.084140,0.161629,0.898554,0.606429,0.009197,0.101472,0.663502,0.005062,0.160808,...,0.030500,0.037348,0.822601,0.360191,0.127061,0.522243,0.769994,0.215821,0.622890,0.085347
3,0.051682,0.531355,0.540635,0.637430,0.726091,0.975852,0.516300,0.322956,0.795186,0.270832,...,0.990505,0.412618,0.372018,0.776413,0.340804,0.930757,0.858413,0.428994,0.750871,0.754543
4,0.103124,0.902553,0.505252,0.826457,0.320050,0.895523,0.389202,0.010838,0.905382,0.091287,...,0.455657,0.620133,0.277381,0.188121,0.463698,0.353352,0.583656,0.077735,0.974395,0.986211
5,0.698162,0.536096,0.309528,0.813795,0.684731,0.162617,0.910927,0.822537,0.949800,0.725720,...,0.138827,0.640875,0.181880,0.345667,0.896788,0.473962,0.667558,0.172320,0.192289,0.040869
6,0.168935,0.278590,0.177010,0.088703,0.120636,0.460779,0.206334,0.364270,0.503417,0.690395,...,0.323679,0.425436,0.507610,0.242410,0.114837,0.610620,0.288631,0.581238,0.154363,0.481140
7,0.532589,0.051824,0.336604,0.134415,0.063375,0.989960,0.322354,0.809874,0.254641,0.681503,...,0.153351,0.586230,0.505889,0.611454,0.018110,0.872124,0.932118,0.565133,0.696651,0.922499
8,0.707239,0.152539,0.576288,0.606715,0.424131,0.736444,0.934367,0.925569,0.450839,0.113238,...,0.722267,0.855696,0.830220,0.397184,0.668085,0.204984,0.293148,0.896336,0.013002,0.085509
9,0.207886,0.026532,0.181435,0.583042,0.421425,0.892672,0.817444,0.341817,0.259423,0.379692,...,0.799416,0.694696,0.272145,0.590231,0.360974,0.091582,0.917314,0.136819,0.950237,0.446006


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.926538,0.382461,0.871469,0.761471,0.328826,0.988821,0.120738,0.358905,0.954462,0.004711,...,0.310465,0.816988,0.930747,0.111477,0.772517,0.801181,0.466825,0.005912,0.705110,0.487674
1,0.715167,0.490948,0.904532,0.319521,0.582585,0.980330,0.019068,0.089363,0.281105,0.143648,...,0.433028,0.132540,0.263659,0.339079,0.234842,0.507921,0.544545,0.197424,0.432392,0.218104
2,0.975796,0.049902,0.092684,0.158453,0.858309,0.652550,0.681106,0.360168,0.843117,0.619341,...,0.156821,0.772316,0.412088,0.796167,0.548580,0.722526,0.141587,0.459266,0.128221,0.661666
3,0.369458,0.911366,0.892686,0.763454,0.581681,0.207756,0.024249,0.925860,0.191849,0.047043,...,0.313598,0.566552,0.844425,0.079068,0.338430,0.921877,0.856621,0.285027,0.505441,0.571166
4,0.794953,0.714644,0.652743,0.639999,0.801813,0.223324,0.468607,0.409739,0.846211,0.488558,...,0.349061,0.986111,0.389271,0.428010,0.645183,0.998789,0.805533,0.310009,0.876316,0.946936
5,0.521674,0.687814,0.343996,0.920405,0.380843,0.211512,0.515157,0.330014,0.169766,0.329920,...,0.323081,0.500177,0.234316,0.847994,0.679627,0.888671,0.584705,0.140129,0.118365,0.108008
6,0.427137,0.959019,0.129011,0.887003,0.701312,0.619073,0.941092,0.891308,0.854728,0.525754,...,0.222091,0.212125,0.862867,0.727157,0.856680,0.461217,0.376074,0.106059,0.768137,0.485547
7,0.810808,0.190487,0.542065,0.050828,0.031726,0.978866,0.059672,0.584194,0.242441,0.160097,...,0.975843,0.019456,0.983457,0.521739,0.554438,0.113520,0.299833,0.281995,0.424159,0.104587
8,0.896924,0.111254,0.593972,0.594881,0.948687,0.545510,0.764649,0.229949,0.990965,0.816547,...,0.573567,0.593088,0.765724,0.683605,0.800564,0.707707,0.608281,0.565341,0.589860,0.967474
9,0.040622,0.588418,0.962138,0.309674,0.181050,0.613000,0.846604,0.852433,0.191767,0.684906,...,0.758048,0.410292,0.488225,0.704057,0.151502,0.453262,0.489614,0.934605,0.039055,0.956150


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.381785,0.884280,0.450580,0.889203,0.400178,0.329899,0.374920,0.289165,0.856012,0.170531,...,0.232548,0.427080,0.687908,0.990223,0.532107,0.291570,0.604532,0.510344,0.178462,0.816248
1,0.575967,0.057404,0.320802,0.174745,0.708598,0.165073,0.852300,0.841379,0.810541,0.867123,...,0.205949,0.183090,0.481792,0.479930,0.360370,0.920427,0.515166,0.698365,0.925812,0.272917
2,0.553476,0.657017,0.721860,0.058866,0.818086,0.882324,0.633707,0.786487,0.107093,0.659608,...,0.725832,0.627375,0.387747,0.204460,0.973627,0.262530,0.912395,0.852041,0.050451,0.668992
3,0.841810,0.738977,0.768721,0.352721,0.454399,0.915650,0.164899,0.872948,0.419942,0.671492,...,0.917378,0.928159,0.034869,0.679377,0.351755,0.233520,0.620001,0.338868,0.797963,0.447284
4,0.069417,0.370450,0.329881,0.882140,0.688254,0.393034,0.288496,0.248113,0.835122,0.668993,...,0.723569,0.378604,0.294903,0.595871,0.940018,0.544825,0.030322,0.157838,0.364742,0.932007
5,0.640042,0.567505,0.986172,0.363177,0.257626,0.219526,0.423415,0.950642,0.622424,0.034000,...,0.069160,0.743788,0.134316,0.357966,0.849020,0.065991,0.809036,0.025196,0.642871,0.582705
6,0.439001,0.354286,0.929918,0.963692,0.967992,0.206173,0.980300,0.623120,0.157829,0.254205,...,0.377401,0.128006,0.233362,0.693897,0.673930,0.717059,0.248278,0.858913,0.219425,0.719200
7,0.391210,0.092130,0.178303,0.442429,0.689687,0.693593,0.737337,0.945797,0.939772,0.048735,...,0.683194,0.685318,0.207105,0.338382,0.154437,0.293999,0.722973,0.188483,0.651001,0.655779
8,0.925906,0.407689,0.267031,0.525947,0.468084,0.944723,0.580411,0.469771,0.326650,0.570644,...,0.238559,0.680445,0.871237,0.260800,0.637075,0.193530,0.532101,0.762386,0.800495,0.412008
9,0.410535,0.316342,0.861576,0.827790,0.708089,0.759283,0.905658,0.153433,0.633961,0.656784,...,0.002756,0.953082,0.882193,0.397721,0.221304,0.557329,0.492624,0.041734,0.559417,0.307542


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.773222,0.022870,0.135256,0.547153,0.112734,0.382484,0.282446,0.479390,0.973054,0.968159,...,0.742549,0.014689,0.638707,0.557382,0.935098,0.161364,0.792444,0.789514,0.522443,0.575358
1,0.608169,0.141071,0.560629,0.028672,0.017801,0.928250,0.939959,0.865063,0.125569,0.062302,...,0.563912,0.085168,0.545653,0.062591,0.079648,0.904816,0.570289,0.112442,0.187270,0.167751
2,0.790280,0.450114,0.316514,0.443655,0.961636,0.183530,0.092308,0.563372,0.137717,0.493172,...,0.943914,0.999072,0.656912,0.879790,0.801385,0.020247,0.274610,0.013139,0.884154,0.128746
3,0.062328,0.129402,0.951153,0.674908,0.706534,0.069130,0.331226,0.421508,0.578126,0.674810,...,0.026716,0.690321,0.373365,0.361318,0.044817,0.219551,0.684745,0.104272,0.996603,0.256260
4,0.139670,0.010372,0.683865,0.662876,0.593358,0.273290,0.001748,0.173523,0.578557,0.084194,...,0.472835,0.914288,0.634008,0.922544,0.063849,0.203463,0.805392,0.097480,0.733605,0.278122
5,0.669848,0.494614,0.202090,0.444083,0.446161,0.456992,0.226185,0.221494,0.696438,0.691768,...,0.626231,0.674574,0.250503,0.413676,0.217465,0.556270,0.768533,0.051927,0.604810,0.324484
6,0.798041,0.958328,0.640620,0.746682,0.316059,0.209726,0.042878,0.032164,0.604368,0.346799,...,0.454389,0.232603,0.710839,0.428112,0.606433,0.767978,0.686680,0.053660,0.512211,0.635196
7,0.554780,0.876255,0.861822,0.855839,0.493228,0.497769,0.041283,0.285226,0.409593,0.229056,...,0.218776,0.153654,0.449319,0.264777,0.468108,0.465911,0.348222,0.218541,0.136806,0.235124
8,0.975532,0.137425,0.967196,0.497551,0.481221,0.595062,0.918240,0.207534,0.507374,0.670020,...,0.280474,0.880227,0.938128,0.277476,0.111782,0.040011,0.945765,0.077110,0.367210,0.919666
9,0.310581,0.717864,0.799290,0.699794,0.210066,0.843294,0.350547,0.578022,0.551745,0.144868,...,0.953911,0.058759,0.205950,0.740339,0.541748,0.029824,0.971199,0.761004,0.484957,0.428933


In [54]:
# 2. 네 개의 DataFrame 모두의 합을 계산해보자 (고전적인 방법)
%timeit df1 + df2 + df3 + df4

1 loop, best of 3: 91.6 ms per loop


In [55]:
# 3. eval를 사용해 계산해보자.
%timeit pd.eval('df1 + df2 + df3 + df4')

10 loops, best of 3: 48.3 ms per loop


> **산술 연산자**

In [56]:
# 고전적인 방법으로 산술연산을 진행
%timeit df1 * df2 / (df3 + df4)

10 loops, best of 3: 96.8 ms per loop


In [57]:
# eval을 사용
%timeit pd.eval('df1 * df2 / (df3 + df4)')

10 loops, best of 3: 49.3 ms per loop


### DaraFrame.query() 메서드

In [58]:
# 테스트 데이터 셋을 생성한다.
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.615875,0.525167,0.047354
1,0.330858,0.412879,0.441564
2,0.689047,0.559068,0.230350
3,0.290486,0.695479,0.852587
4,0.424280,0.534344,0.245216


In [59]:
# 1-1. 0.5보다 작은 데이터를 찾는 고전적인 방법
df[(df.A < 0.5) & (df.B < 0.5)].head()

,A,B,C
1,0.330858,0.412879,0.441564
8,0.448611,0.415924,0.481001
10,0.112910,0.394884,0.950129
11,0.191011,0.118751,0.130223
14,0.075723,0.260648,0.956146


In [60]:
# 1-2. 0.5보다 작은 데이터를 찾기 위해 Query를 사용
df.query('A < 0.5 and B < 0.5').head()

,A,B,C
1,0.330858,0.412879,0.441564
8,0.448611,0.415924,0.481001
10,0.112910,0.394884,0.950129
11,0.191011,0.118751,0.130223
14,0.075723,0.260648,0.956146


In [61]:
# 2-1 고전적인 방법
Cmean = df['C'].mean()
display(Cmean)

df[(df.A < Cmean) & (df.B < Cmean)].head()

0.5104765814526574

,A,B,C
1,0.330858,0.412879,0.441564
8,0.448611,0.415924,0.481001
10,0.112910,0.394884,0.950129
11,0.191011,0.118751,0.130223
14,0.075723,0.260648,0.956146


In [62]:
# 2-2 Query를 사용
df.query('A < @Cmean and B < @Cmean').head()

,A,B,C
1,0.330858,0.412879,0.441564
8,0.448611,0.415924,0.481001
10,0.112910,0.394884,0.950129
11,0.191011,0.118751,0.130223
14,0.075723,0.260648,0.956146


`eval`이나 `query`는 언제 써야 할까? 메모리를 절약하는데 사용한다. 작은 배열에서는 오히려 전형적인 메서드가 더 빠르다.